In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import src.config as config

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/423065
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view already existed. Skip creation.


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using ArrowFlight (16.51s) 


In [6]:
# drop `pickup_ts` column
ts_data.drop('pickup_ts', axis=1, inplace=True)

# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
1342995,2023-01-01 00:00:00+00:00,0,1
598467,2023-01-01 01:00:00+00:00,0,1
101224,2023-01-01 02:00:00+00:00,0,1
31729,2023-01-01 03:00:00+00:00,0,1
513079,2023-01-01 04:00:00+00:00,0,1
...,...,...,...
1719676,2024-02-28 00:00:00+00:00,0,265
1719958,2024-02-28 01:00:00+00:00,8,265
1720098,2024-02-28 02:00:00+00:00,3,265
1720473,2024-02-28 03:00:00+00:00,0,265


In [7]:
# from src.plot import plot_ts
from typing import Optional, List
import pandas as pd
import plotly.express as px 

def plot_ts(
    ts_data: pd.DataFrame,
    locations: Optional[List[int]] = None
    ):
    """
    Plot time-series data
    """
    ts_data_to_plot = ts_data[ts_data.pickup_location_id.isin(locations)] if locations else ts_data

    fig = px.line(
        ts_data_to_plot,
        x="pickup_hour",
        y="rides",
        color='pickup_location_id',
        template='none',
    )

    fig.show()

plot_ts(ts_data, locations=[43])

In [8]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [00:34<00:00,  7.78it/s]


features_and_target.shape=(99941, 675)


In [9]:
features_and_target

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-01-29 00:00:00+00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,0.0,...,1.0,2.0,1.0,1.0,0.0,0.0,0.0,2023-01-29 23:00:00+00:00,1,0.0
2,2.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,2.0,1.0,2.0,0.0,1.0,0.0,2023-01-30 22:00:00+00:00,1,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,2.0,0.0,0.0,0.0,2023-01-31 21:00:00+00:00,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,1.0,0.0,2.0,1.0,1.0,0.0,2023-02-01 20:00:00+00:00,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99936,1.0,3.0,2.0,3.0,1.0,5.0,0.0,2.0,1.0,1.0,...,3.0,5.0,6.0,8.0,6.0,0.0,2.0,2024-02-24 07:00:00+00:00,265,3.0
99937,1.0,1.0,0.0,3.0,2.0,0.0,3.0,4.0,3.0,1.0,...,2.0,4.0,2.0,3.0,3.0,7.0,3.0,2024-02-25 06:00:00+00:00,265,1.0
99938,0.0,0.0,2.0,2.0,4.0,1.0,2.0,5.0,0.0,1.0,...,1.0,5.0,5.0,0.0,3.0,2.0,1.0,2024-02-26 05:00:00+00:00,265,2.0
99939,2.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,1.0,1.0,...,2.0,9.0,2.0,1.0,2.0,1.0,0.0,2024-02-27 04:00:00+00:00,265,1.0


In [10]:
from datetime import date, timedelta
from pytz import timezone
import pandas as pd
from src.data_split import train_test_split

# training data -> from January 2022 up until 2 months ago
# test data -> last 2 months
cutoff_date = pd.to_datetime(date.today() - timedelta(days=28*1), utc=True)

print(f'{cutoff_date=}')

X_train, y_train, X_test, y_test = train_test_split(
    features_and_target,
    cutoff_date,
    target_column_name='target_rides_next_hour'   
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

cutoff_date=Timestamp('2024-01-30 00:00:00+0000', tz='UTC')
X_train.shape=(91863, 674)
y_train.shape=(91863,)
X_test.shape=(8078, 674)
y_test.shape=(8078,)


In [11]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
    
    # sort X_train by `pikup_hour` inplace
    # so the TimeSeriesSplit will split the data in a consistent way
    X_train.sort_values('pickup_hour', inplace=True)

    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [12]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

[I 2024-02-27 21:58:31,027] A new study created in memory with name: no-name-edc6196d-ec59-4975-aeb1-6d90903d6022
C:\Users\micha\taxi_demand_predictor_ml2\src\model.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\micha\taxi_demand_predictor_ml2\src\model.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\micha\taxi_demand_predictor_ml2\src\model.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [13]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 19, 'feature_fraction': 0.5535453243593246, 'bagging_fraction': 0.24794648125877955, 'min_child_samples': 86}


In [14]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.5535453243593246, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5535453243593246
[LightGBM] [Warning] bagging_fraction is set=0.24794648125877955, subsample=1.0 will be ignored. Current value: bagging_fraction=0.24794648125877955
[LightGBM] [Warning] feature_fraction is set=0.5535453243593246, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5535453243593246
[LightGBM] [Warning] bagging_fraction is set=0.24794648125877955, subsample=1.0 will be ignored. Current value: bagging_fraction=0.24794648125877955
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.231707 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171896
[LightGBM] [Info] Number of data points in the train set: 91863, number of used features: 676
[LightGBM] [Info] Start training from 

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000001D1B4423B80>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.24794648125877955,
                               feature_fraction=0.5535453243593246,
                               min_child_samples=86, num_leaves=19))])

In [15]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.5535453243593246, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5535453243593246
[LightGBM] [Warning] bagging_fraction is set=0.24794648125877955, subsample=1.0 will be ignored. Current value: bagging_fraction=0.24794648125877955
test_mae=30.0876


In [16]:
import joblib
from src.paths import MODELS_DIR

joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['C:\\Users\\micha\\taxi_demand_predictor_ml2\\models\\model.pkl']

In [17]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [18]:
model_registry = project.get_model_registry()

model = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor with a bit of hyper-parameter tuning",
    input_example=X_train.sample(),
    model_schema=model_schema
)

model.save(MODELS_DIR / 'model.pkl')

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://c.app.hopsworks.ai:443/p/423065/models/taxi_demand_predictor_next_hour/4


Model(name: 'taxi_demand_predictor_next_hour', version: 4)